# Построение движка для игр на pygame - урок 8

## Повторение

Рассмотрим решение домашнего задания

In [2]:
from first_engine.game import Game
from first_engine import objects
import pygame


class FirstGame(Game):
    def run(self):
        hero = objects.Character(self.surface, speed=4)
        stand_right = pygame.image.load('./sprites/viking/stand/right/1.png')
        stand_left = pygame.image.load('./sprites/viking/stand/left/1.png')
        run_right = pygame.image.load('./sprites/viking/run/right/1.png')  # добавляем картинку с бегом
        run_left = pygame.image.load('./sprites/viking/run/left/1.png')  # добавляем картинку с бегом
        attack_right = pygame.image.load('./sprites/viking/attack1/right/1.png') # добавляем картинку с атакой
        attack_left = pygame.image.load('./sprites/viking/attack1/left/1.png') # добавляем картинку с атакой
        attack = 0 # объявляем атаку

        side = 'right' # объявляем сторону

        game_over = False
        while self.RUNNER:
            if not game_over:
                self.cycle_init(objects=[hero], FPS=60)

                sides = hero.motion_control()  # сохраняем возвращаемые значения

                hero.blit()  # для отладки

                if attack: # если атака есть
                    if side == 'right': # если вправо
                        self.surface.blit(attack_right, hero.body)
                    elif side == 'left': # если влево
                        self.surface.blit(attack_left, hero.body)
                    attack -= 1
                elif sum(sides.values()) == 0 and side == 'right': # если вправо
                    self.surface.blit(stand_right, hero.body)
                elif sum(sides.values()) == 0 and side == 'left': # если влево
                    self.surface.blit(stand_left, hero.body)
                elif sides['right']:
                    side = 'right'
                    self.surface.blit(run_right, hero.body)
                elif sides['left']:
                    side = 'left'
                    self.surface.blit(run_left, hero.body)

            for event in self.events():
                # print(event)  # отслеживание событий
                if event.type == 768 and event.key == 102: # если нажата кнопка F
                    attack = 15 # объявляем число кадров атаки
                self.close(event)


FirstGame(width=600, height=400).run()

## Внутренняя анимация

Давайте попробуем добавить анимацию внутри одного действия персонажа

In [ ]:
from first_engine.game import Game
from first_engine import objects
import pygame
import os


class FirstGame(Game):
    def run(self):
        hero = objects.Character(self.surface, speed=8)

        stand = './sprites/viking/stand/right/' # возьмем путь до всех спрайтов в одной папке
        stand = [pygame.image.load(stand + i) for i in os.listdir(stand)] # сделаем список загруженных спрайтов

        time_to_update_sprites = 6 # назначаим время до смены спрайта
        sprite_id = 0 # начинаем с 1го спрайта в каждом списке

        game_over = False
        while self.RUNNER:
            if not game_over:
                self.cycle_init(objects=[hero], FPS=60)

                sides = hero.motion_control()
                
                time_to_update_sprites -= 1     # запускаяем обратный отсчет до обновления спрайта
                if time_to_update_sprites == 0: # если время до обновления = 0
                    sprite_id += 1              # тогда переключаемся на следующий спрайт
                    time_to_update_sprites = 6  # обновляем счетчик до обновления
                    if sprite_id == len(stand): # если id спрайта равна длине спискаспрайтов
                        sprite_id = 0           # начинаем снова с первого спрайта

                if sum(sides.values()) == 0: # если не нажата ни одна кнопка движения
                    self.surface.blit(stand[sprite_id], hero.body) # тогда отрисовываем спрайты из группы stand по очереди

            for event in self.events():
                # print(event)  # отслеживание событий
                self.close(event)


FirstGame(width=600, height=400).run()

# Задание:
- Посмотрите на код и на комментарии, и попробуйте определить, на сколько методов можно было бы разбить этот код, и за что бы они отвечали
- Подумайте какого действия здесь не хватает

Давайте допишем нашу функцию __load_sprites__ в классе __Objects__

In [ ]:
class Object:
    sprites = {} # добавим словарь для хранения спрайтов

    def __init__(self, parent: pygame.Surface, width=40, height=40, x=0, y=0, color=(255, 255, 255)):...

    def load_sprites(self, name: str, path: str, update: int = 6):
        """
        Метод для загрузки спрайтов в sprites
        :param update: частота обновления спрайта
        :param name: str - имя под которым будем хранить загруженные скины и созданные тела, необходимые именя:
        run_left, run_right, stand_left, stand_right, jump_left, jum_right, attack_left, attack_right
        :param path: str - путь до папки с файлами
        :return skins: dict - возвращаем только что загруженные скины и созданные тела
        """
        skins = [pygame.image.load(path + i) for i in os.listdir(path)]

        self.sprites[name] = {
            'skins': skins, # Сохраняем загруженные спрайты
            'update': update # Для этих спрайтов сохраням частоту обновлений спрайтов
        }
        return skins

    def blit(self):...

    def recolor(self, color):...

    def replace(self, x=None, y=None):...

Загруженные спрайты мы будем отрисовывать по очереди, а так же делать на их основе хитбоксы.<br>
Для этого давайте создадим метод __remake_for_skin__ в __Object__.

In [ ]:
class Object:
    sprites = {}
    sprite_id = 0 # добавим переменную, указывающую на конкретный спрайт

    def __init__(self, parent: pygame.Surface, width=40, height=40, x=0, y=0, color=(255, 255, 255)):...
        
    def blit(self):
        # от сюда удаляем верхнюю строчку, так как она нам больше не нужна
        self.parent.blit(source=self.skin, dest=self.body)
        return self.parent
    
    def recolor(self, color):...

    def load_sprites(self, name: str, path: str) -> list[pygame.Surface]:...

    def __remake_for_skin(self, name: str):
        """
        Метод для переделывания скина и тела по индексу скина и его названия в sprites
        :param name: str название скинов
        :return: pygame.Surface
        """
        self.skin = self.sprites[name]['skins'][self.sprite_id]
        self.body = self.skin.get_rect(topleft=(self.body.x, self.body.y))
        return self.skin

Теперь для этого метода нам нужно сделать еще один - который будет отсчитывать время до смены спрайта, на основе ключа __update__ и дли массива текущих спрайтов __skins__, назовем его __sprite_update__

In [ ]:
class Object:
    sprites = {}

    time_to_sprite_update = 6 # переменная отвечающая за время до смены спрайта
    sprite_id = 0

    def __init__(self, parent: pygame.Surface, width=40, height=40, x=0, y=0, color: tuple = (255, 255, 255)):...

    def blit(self):...

    def recolor(self, color):...

    def load_sprites(self, name: str, path: str, update: int = 6):...

    def __sprite_update(self, name):
        """
        Метод обновляющий спрайты по очереди
        :param name: str - принимает имя спрайтов сохраненных в словаре sprites
        :return: int - возвращает id текущего спрайта
        """
        self.time_to_sprite_update -= 1
        if self.time_to_sprite_update == 0:
            self.sprite_id += 1
            self.time_to_sprite_update = self.sprites[name]['update']
            if self.sprite_id == len(self.sprites[name]['skins']):
                self.sprite_id = 0
        return self.sprite_id

    def __remake_for_skin(self, name: str):...

Остался последний метод для проигрывания анимации - так его и назовем __play_animation__ - он будет высывать два предыдущих метода по очереди, и его мы будем вызывать, для проигрывания анимации персонажа

In [ ]:
class Object:
    sprites = {}
    
    time_to_sprite_update = 6
    sprite_id = 0
    
    action = None # переменная принимающая последнее действие - служит для запуска анимации последнего действия
    actions = {} # здесь будем хранить все возможные действия для обекта созданного на основе данного класса
    
    def __init__(self, parent: pygame.Surface, width=40, height=40, x=0, y=0, color: tuple = (255, 255, 255)):...

    def blit(self):...

    def recolor(self, color):...

    def load_sprites(self, name: str, path: str, update: int = 6):...

    def __sprite_update(self, name):...

    def __remake_for_skin(self, name: str):...
        
    def play_animation(self):
        self.__sprite_update(self.action)
        self.__remake_for_skin(self.action)

Мы подготовили все для проигрывания анимации, но мы уже не создаем объекты на основе класса __Object__, и особенности разных объектов будут описываться в дочерних классах, перейдем к ним

Нам нужно модифицировать __motion_control__ - он больше не будет отвечать за перемещение персонажа, но будет отвечать просто за отслеживание событий<br>
А сами перемещения разобъем на другие методы

In [ ]:
class Character(Object):
    """
    Класс для создания персонажей
    """

    def __init__(self, parent: pygame.Surface, width=40, height=40, x=0, y=0, color=(255, 255, 255), speed=1):
        """
        :param parent: pygame.Surface - родимтельское окно
        :param width: int
        :param height: int
        :param x: int
        :param y: int
        :param color: tuple - цвет в RGB
        :param speed: int - скорость персонажа
        """
        super().__init__(parent, width, height, x, y, color)
        self.speed = speed

        self.action = 'stand_right' # по умолчанию наш персонаж будет применять анимацию стояния вправо

    def __motions(self): # переименуем motion_control просто в motion
        left = pygame.key.get_pressed()[97]
        right = pygame.key.get_pressed()[100]
        up = pygame.key.get_pressed()[119]
        down = pygame.key.get_pressed()[115]

        stand = not(left + right + up + down)
        if stand and 'left' in self.action:
            self.action = 'stand_left'
        elif stand and 'right' in self.action:
            self.action = 'stand_right'

        self.actions = {'left': left, 'right': right, 'up': up, 'down': down, 'default': stand}

        return self.actions

    def motion_left(self):
        """
        Метод для движения влево, можно подключить загруженные спрайты
        :return: int - скорость
        """
        left = self.__motions()['left']
        if left:
            self.body.x -= left * self.speed
            self.action = 'run_left'
        return left

    def motion_right(self):
        """
        Метод для движения вправо, можно подключить загруженные спрайты
        :return: int - скорость
        """
        right = self.__motions()['right']
        if right:
            self.body.x += right * self.speed
            self.action = 'run_right'
        return right

    def motion_up(self):
        """
        Метод для движения вверх, можно подключить загруженные спрайты
        :return: int - скорость
        """
        up = self.__motions()['up']
        if up:
            self.body.y -= up * self.speed
        return up

    def motion_down(self):
        """
        Метод для движения вниз, можно подключить загруженные спрайты
        :return: int - скорость
        """
        down = self.__motions()['down']
        if down:
            self.body.y += down * self.speed
        return down

Мы облегчили себе задачу по использованию спрайтов, теперь она выглядит вот так:

In [ ]:
from first_engine.game import Game
from first_engine import objects


class FirstGame(Game):
    def run(self):
        hero = objects.Character(self.surface, speed=8)
        hero.load_sprites(name='stand_right', path='./sprites/viking/stand/right/')
        hero.load_sprites(name='stand_left', path='./sprites/viking/stand/left/')
        hero.load_sprites(name='run_left', path='./sprites/viking/run/left/')
        hero.load_sprites(name='run_right', path='./sprites/viking/run/right/')

        game_over = False
        while self.RUNNER:
            if not game_over:
                self.cycle_init(FPS=60)

                # Объекты
                hero.blit()
                hero.play_animation()

                # Движения персонажа
                hero.motion_left()
                hero.motion_right()

                # проверяем - уперся ли персонаж в преграду
                self.window_borders([hero])

            for event in self.events():
                # print(event)  # отслеживание событий
                self.close(event)


FirstGame(width=800, height=600).run()

# Задание:
- Опишите логику прыжка и падения персонажа